In [5]:
import math
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.

This is by no means production code.
"""
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
import requests

# constants
BASE_URL = "https://www.domain.com.au"

# begin code
url_links = []
property_metadata = defaultdict(dict)
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
# generate list of urls to visit
regions = ["bendigo-greater-area-vic","shepparton-greater-area-vic","macedon-ranges-greater-area-vic","mildura-greater-area-vic",\
    "monash-city-greater-area-vic","wyndham-city-council-greater-area-vic"]
for region in regions:
    url = BASE_URL + f"/rent/{region}/?sort=price-desc&page=1"
    bs_object = BeautifulSoup(requests.get(url, headers=headers).text, "html.parser")
    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    result_count = int(bs_object \
        .find(
            "h1",
            {"data-testid": "summary","class":"css-ekkwk0"}
        ).text.split()[0])
    if result_count < 1000:
        num_pages = range(1,math.ceil(result_count/20))
    else:
        num_pages = range(1,50)
    print(region, result_count,num_pages)
    for page in num_pages:
        url = BASE_URL + f"/rent/{region}/?sort=price-desc&page={page}"
        bs_object = BeautifulSoup(requests.get(url, headers=headers).text, "html.parser")

        # find the unordered list (ul) elements which are the results, then
        # find all href (a) tags that are from the base_url website.
        index_links = bs_object \
            .find(
                "ul",
                {"data-testid": "results"}
            ) \
            .findAll(
                "a",
                href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
            )

        for link in index_links:
            # if its a property address, add it to the list
            if 'address' in link['class']:
                url_links.append(link['href'])


bendigo-greater-area-vic 209 range(1, 11)
shepparton-greater-area-vic 81 range(1, 5)
macedon-ranges-greater-area-vic 53 range(1, 3)
mildura-greater-area-vic 104 range(1, 6)
monash-city-greater-area-vic 468 range(1, 24)
wyndham-city-council-greater-area-vic 593 range(1, 30)


In [6]:
len(url_links)

1460

In [7]:
# for each url, scrape some basic metadata
count = 0
for property_url in url_links[1:]:
    bs_object = BeautifulSoup(requests.get(property_url, headers=headers).text, "html.parser")
    # looks for the header class to get property name
    property_metadata[property_url]['name'] = bs_object \
        .find("h1", {"class": "css-164r41r"}) \
        .text
    property_metadata[property_url]['type'] = bs_object \
        .find("div", {"data-testid": "listing-summary-property-type"}) \
        .text
    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) \
        .text

    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object \
                .find(
                    "a",
                    {"target": "_blank", 'rel': "noopener noreferer"}
                ) \
                .attrs['href']
        )[0].split(',')
    ]

    property_metadata[property_url]['rooms'] = [
        re.findall(r'\d\s[A-Za-z]+', feature.text) for feature in bs_object \
            .find("div", {"data-testid": "property-features"}) \
            .findAll("span", {"data-testid": "property-features-text-container"})
    ]

    count +=1
    print(count)


# output to example json in data/raw/
with open('../data/raw/domain_scrap_areas.json', 'w') as f:
    dump(property_metadata, f)



    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
